In [1]:
import mne
import pickle
import numpy as np
import matplotlib.pylab as plt
import os.path as op
from tools import files
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
data_folder = "/cubric/scratch/c1557187/stirring_source/data"

In [3]:
all_participants = files.get_folders_files(data_folder, wp=False)[0]
garbage = ["001", "043", "042"]
participants = [i for i in all_participants if i not in garbage]

In [6]:
output = {str(i): [] for i in range(1,9)}

for i in participants:
    data_path = op.join(data_folder, i, "filtered_01_40")
    epoch_paths = files.get_files(data_path, "ph2", "epo.fif")[2]
    epoch_paths.sort()
    fwd_paths = files.get_files(op.join(data_folder, i), "", "fwd.fif")[2]
    fwd_paths = [i for i in fwd_paths if "rs" not in i]
    fwd_paths.sort()
    noise_paths = files.get_files(data_path, "mx", "cov.fif")[2]
    noise_paths = [i for i in noise_paths if "rs" not in i]
    noise_paths.sort()
#     src_path = files.get_files(op.join(data_folder, i), i, "src.fif")[2][0]
#     src = mne.read_source_spaces(src_path)
    
    for e, f, n in zip(epoch_paths, fwd_paths, noise_paths):
#         print(e, f, n)
        epo = mne.read_epochs(e)
        fwd = mne.read_forward_solution(f)
        noise = mne.read_cov(n)
        inv = mne.minimum_norm.make_inverse_operator(epo.info, fwd, noise, depth=None, fixed=False)
        lambda2 = 1.0 / 3.0 ** 2
        for k in output.keys():
            avg_epo = epo[k].average()
            stc = mne.minimum_norm.apply_inverse(avg_epo, inv, lambda2, "dSPM", pick_ori=None)
            morph = mne.compute_source_morph(stc, subject_from=i, subject_to="fsaverage", subjects_dir="/cubric/data/c1557187/stirring_MEG/mri/fs_results")
            stc_fs = morph.apply(stc)
            output[k].append(stc_fs)       

In [7]:
filename = op.join(data_folder, "01_40_avg_ph2.pickle")
with open(filename, "wb") as f:
    pickle.dump(output, f)